In [ ]:
import torchaudio.transforms as T
import torch
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# 가상의 멜 스펙트로그램 변환 함수
mel_transform = T.MelSpectrogram(sample_rate=8000, n_fft=400, hop_length=256, n_mels=64)

# TSV 파일 불러오기 (xmin, ymin은 이미 존재하는 데이터)
df = pd.read_csv('your_data.tsv', sep='\t', header=None, names=['xmin', 'ymin', 'label_ind'])

# 가상의 오디오 파일을 로드하고 해당 구간의 멜 스펙트로그램 계산
audio_file = 'your_audio_file.wav'  # 실제 데이터에 맞게 수정
audio_waveform, _ = torchaudio.load(audio_file, normalize=True)

start_sample = int(df['xmin'].iloc[0])  # 예시로 첫 번째 행의 xmin을 사용
end_sample = int(df['ymin'].iloc[0])  # 예시로 첫 번째 행의 ymin을 사용

audio_segment = audio_waveform[:, start_sample:end_sample]
mel_spec = mel_transform(audio_segment)

# ymin을 멜 스펙트로그램의 최대값으로 설정
ymin_max = mel_spec.max().item()
df['ymin'] = ymin_max

# 최종 TSV 파일로 저장
df.to_csv('your_output_data.tsv', sep='\t', index=False, header=False)

# 멜 스펙트로그램 시각화
plt.figure(figsize=(10, 5))
plt.imshow(mel_spec[0].numpy(), cmap='viridis', aspect='auto', origin='lower')

# bbox 추가
for idx, row in df.iterrows():
    xmin = row['xmin']
    ymin = row['ymin']
    xmax = row['xmax']  # 미리 계산된 xmax를 사용해야 합니다.
    ymax = row['ymax']  # 미리 계산된 ymax를 사용해야 합니다.

    width = xmax - xmin
    height = ymax - ymin

    # 붉은색 bbox 추가
    rect = patches.Rectangle((xmin, ymin), width, height, linewidth=1, edgecolor='red', facecolor='none')
    plt.gca().add_patch(rect)

plt.ylim(0, mel_spec.shape[1])
plt.xlim(0, mel_spec.shape[2])
plt.show()
